### Build a list with unit dictionaries including below data
- {
- 'musicname':'original filename',
- 'staff_id':int,
- 'start':int,
- 'end':int,
- 'tempo':int,
- 'timesig':'4/4' or '6/8' or...,
- 'beats':'1/4,0; 1/2,1; 1/8,1; 1/8,0' (假設為4/4拍，左為一單位的拍子，順序為：四分休止符、二分音符、八分音符、八分休止符，1/4 + 1/2 + 1/8 + 1/8 = 1)
- }

In [74]:
from bs4 import BeautifulSoup as bs4
import os

#Determine tempo, start measure & end measure
def getTempoMeasure(dirPath, filename, mainList):
    filePath = dirPath + filename
    with open(filePath, 'r') as f:
        mscx = bs4(f.read(), 'xml')
    
    #Select measures of staff#1 because tempos only appear in staff#1.
    staff1Measures = mscx.select('Score > Staff:nth-of-type(1) > Measure')
    
    #Get the 1st tempo.
    tempo = staff1Measures[0].select('Tempo > text')[0].text.rsplit(' ', 1)[-1]
    
    #Set tempos, measure numbers, & filename to the list.
    filename = filename.rsplit('.', 1)[0]
    print filename
    tmp = []
    for i, m in enumerate(staff1Measures[:-1], 1):
        for t in m.select('Tempo > text'):
            tmp.append(int(t.text.rsplit(' ', 1)[-1]))
        if i % 4 == 0:
            if tmp:
                tempo = max(tmp)
            mainList.append(genDataDic(tempo, i-3, i, filename))
            tmp[:] = []
    
    #Handle last measure different situations with for-else.
    else:
        i += 1
        for t in staff1Measures[-1].select('Tempo > text'):
            tmp.append(int(t.text.rsplit(' ', 1)[-1]))
        if tmp:
            tempo = max(tmp)    
        if i % 4 == 1: 
            mainList.append(genDataDic(tempo, i, i, filename))
        elif i % 4 == 2:
            mainList.append(genDataDic(tempo, i-1, i, filename))
        elif i % 4 == 3:
            mainList.append(genDataDic(tempo, i-2, i, filename))
        else:
            mainList.append(genDataDic(tempo, i-3, i, filename))

    for e in mainList:
        print e

def genDataDic(tempo, start, end, filename):
    return {'tempo':tempo, 'start':start, 'end':end, 'musicname':filename}

In [ ]:
#Determine tempo, start measure & end measure
mainList = []
dirPath = 'C:/Users/BigData/Desktop/mscx/'
for filename in os.listdir(dirPath):
    getTempoMeasure(dirPath, filename, mainList)

In [81]:
from bs4 import BeautifulSoup as bs4
from decimal import *
import os

#The beat dictionary.
beatDic = {'measure':None,
           'whole':1,
           'half':Decimal(1)/2,
           'quarter':Decimal(1)/4,
           'eighth':Decimal(1)/8,
           '16th':Decimal(1)/16,
           '32nd':Decimal(1)/32,
           '64th':Decimal(1)/64,
           '128th':Decimal(1)/128,
           '256th':Decimal(1)/256
          }

#Determine time signature & beats
#Note: About <tick><track> tags indicating at least
#1 additional melody in a staff, omit them at first.
def getBeat(dirPath, filename, mmTuple, mainList2):
    filePath = dirPath + filename
    with open(filePath, 'r') as f:
        mscx = bs4(f.read(), 'xml')
    
    #Get 1st time signature
    staff1TimeSig = mscx.select_one('Score > Staff:nth-of-type(1) TimeSig')
    sigN = int(staff1TimeSig.find('sigN').text)
    sigD = int(staff1TimeSig.find('sigD').text)
    beatDic['measure'] = Decimal(sigN)/sigD
    diffTimeSigs = False
    
    #Set staff_id, time signature & beats to the list
    for mm in mmTuple:
        for i, m in enumerate(mscx.select('Score > \
                              Staff:nth-of-type('+str(mm)+') > \
                              Measure')[:-1], 1):
            print 'measure #' + str(i)
            if m.find('TimeSig'):
                sigN = int(m.find('sigN').text)
                sigD = int(m.find('sigD').text)
                if Decimal(sigN)/sigD != beatDic['measure']:
                    beatDic['measure'] = Decimal(sigN)/sigD
                    if i % 4 != 2 or i % 4 == 3:
                        diffTimeSigs = True
                        continue
                    if i % 4 == 0:
                        diffTimeSigs = False
                        continue
                        
#             for c in m.children:
#                 if str(type(c)) == '<class \'bs4.element.Tag\'>':
#                     if c.name == 'Chord':
#                         print c.name
#                     elif c.name == 'Rest':
#                         print c.name

            if i % 4 == 0:
                if diffTimeSigs == True:
                    diffTimeSigs = False
                    continue
                timeSig = str(sigN)+'/'+str(sigD)
                mainList2.append(genDataDic2(mm, timeSig))

        #Handle last measure different situations with for-else.
        else:
            i += 1
#             for t in staff1Measures[-1].select('Tempo > text'):
#             tmp.append(int(t.text.rsplit(' ', 1)[-1]))
            mscx.select('Score > \
                              Staff:nth-of-type('+str(mm)+') > \
                              Measure')[:-1]
            print 'measure #' + str(i)
            if m.find('TimeSig'):
                sigN = int(m.find('sigN').text)
                sigD = int(m.find('sigD').text)
                if Decimal(sigN)/sigD != beatDic['measure']:
                beatDic['measure'] = Decimal(sigN)/sigD
            if i % 4 == 1:
                mainList2.append(genDataDic2(mm))
            elif i % 4 == 2:
                mainList2.append(genDataDic2(mm))
            elif i % 4 == 3:
                mainList2.append(genDataDic2(mm))
            else:
                mainList2.append(genDataDic2(mm))

    for e in mainList2:
        print e

def genDataDic2(staff_id, timeSig):
    return {'staff_id':staff_id, 'timesig':timeSig}
                                 
#Staffs which play main melodies
mmTuple = (1, 2)

#Determine time signature & beats
mainList2 = []
dirPath = 'C:/Users/BigData/Desktop/mscx/'
for filename in os.listdir(dirPath):
    getBeat(dirPath, filename, mmTuple, mainList2)


{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1, 'timesig': '6/8'}
{'staff_id': 1